In [1]:
#import modules
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [2]:
chess_data = pd.read_csv('./datacleaned.csv')

In [3]:
chess_data = chess_data.drop(chess_data.columns[[0,2]], axis=1)

In [4]:
chess_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19113 entries, 0 to 19112
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   black_rating    19113 non-null  int64 
 1   rated           19113 non-null  bool  
 2   turns           19113 non-null  int64 
 3   victory_status  19113 non-null  object
 4   white_rating    19113 non-null  int64 
 5   winner          19113 non-null  object
dtypes: bool(1), int64(3), object(2)
memory usage: 765.4+ KB


In [5]:
dummy = pd.get_dummies(chess_data['victory_status'])

In [6]:
chess_with_dummy = pd.concat([chess_data, dummy], axis=1)
chess_with_dummy.head()

,black_rating,rated,turns,victory_status,white_rating,winner,draw,mate,outoftime,resign
0,1191,False,13,outoftime,1500,white,0,0,1,0
1,1261,True,16,resign,1322,black,0,0,0,1
2,1500,True,61,mate,1496,white,0,1,0,0
3,1454,True,61,mate,1439,white,0,1,0,0
4,1469,True,95,mate,1523,white,0,1,0,0


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [8]:
scaler= StandardScaler()
scaler.fit(chess_with_dummy[['black_rating']])
print(scaler.mean_)
chess_with_dummy['black_ss'] = scaler.transform(chess_with_dummy[['black_rating']])

[1590.04551876]


In [9]:
scaler.fit(chess_with_dummy[['white_rating']])
print(scaler.mean_)
chess_with_dummy['white_ss'] = scaler.transform(chess_with_dummy[['white_rating']])
scaler.fit(chess_with_dummy[['turns']])
print(scaler.mean_)
chess_with_dummy['turns_ss'] = scaler.transform(chess_with_dummy[['turns']])

[1597.30000523]
[60.51383875]


In [28]:
#y = chess_with_dummy.pop('winner')
X = chess_with_dummy.drop(columns=['victory_status'])
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19113 entries, 0 to 19112
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   black_rating  19113 non-null  int64  
 1   rated         19113 non-null  bool   
 2   turns         19113 non-null  int64  
 3   white_rating  19113 non-null  int64  
 4   draw          19113 non-null  uint8  
 5   mate          19113 non-null  uint8  
 6   outoftime     19113 non-null  uint8  
 7   resign        19113 non-null  uint8  
 8   black_ss      19113 non-null  float64
 9   white_ss      19113 non-null  float64
 10  turns_ss      19113 non-null  float64
dtypes: bool(1), float64(3), int64(3), uint8(4)
memory usage: 989.4 KB


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
#finished loading train test data


In [43]:
#Logistic Regression
import seaborn as sns
import numpy as np
import sklearn.metrics as m

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)
print('Accuracy of Logistic Regression: {}'.format(m.accuracy_score(y_test,lr_y_pred)))
print('Precision of Logistic Regression: {}'.format(m.precision_score(y_test,lr_y_pred, average='micro')))
print('Recall of Logistic Regression: {}'.format(m.recall_score(y_test,lr_y_pred)))
print('F1 of Logistic Regression: {}'.format(m.f1_score(y_test,lr_y_pred)))
cf_matrix = confusion_matrix(y_test, lr_y_pred)
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, cmap='Blues', fmt='.4%')

Accuracy of Logistic Regression: 0.656552445723254
Precision of Logistic Regression: 0.656552445723254


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].